In [ ]:
!pip install backtesting

In [ ]:
!pip install yfinance

In [3]:
import pandas as pd
from backtesting import Strategy

/usr/local/lib/python3.9/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


# Get Data

In [4]:
from backtesting.test import GOOG

GOOG.head()

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


# Technical Indicator

In [5]:
from backtesting.test import SMA
from backtesting.lib import crossover

class SmaCross(Strategy):
    n1 = 20 # period of the first SMA
    n2 = 50 # period of the second SMA

    def init(self):
        close = self.data.Close # close price data
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)
      
    def next(self):
      if crossover(self.sma1, self.sma2):
          self.buy()
      elif crossover(self.sma2, self.sma1):
          self.sell()

# Mean Reversion Lean Hogs

In [6]:
import yfinance as yf

# Obtain OHLV data for HE
he = yf.download("HE", start="2023-01-15", interval="15m")[
    ["Open", "High", "Low", "Close", "Volume"]
]
he.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2023-01-17 09:30:00,42.310001,42.430000,42.310001,42.424999,859
2023-01-17 09:45:00,42.410000,42.514999,42.410000,42.450001,3079
2023-01-17 10:00:00,42.445000,42.520000,42.445000,42.520000,17368
2023-01-17 10:15:00,42.529999,42.535000,42.450001,42.490002,6055
2023-01-17 10:30:00,42.480000,42.480000,42.320000,42.360001,3936


In [7]:
from backtesting.test import SMA

def std_3(arr, n):
    return pd.Series(arr).rolling(n).std() * 3

class MeanReversion(Strategy):
    roll = 50

    def init(self):
        self.he = self.data.Close

        self.he_mean = self.I(SMA, self.he, self.roll)
        self.he_std = self.I(std_3, self.he, self.roll)
        self.he_upper = self.he_mean + self.he_std
        self.he_lower = self.he_mean - self.he_std

        self.he_close = self.I(SMA, self.he, 1)

    def next(self):

        if self.he_close < self.he_lower:
            self.buy(
                tp = self.he_mean,
            )

        if self.he_close > self.he_upper:
            self.sell(
                tp = self.he_mean,
            )

from backtesting import Backtest

bt = Backtest(he, MeanReversion, cash=10000, commission=0.002)
stats = bt.run()
bt.plot()
stats

Start                     2023-01-17 09:30:00
End                       2023-03-08 15:45:00
Duration                     50 days 06:15:00
Exposure Time [%]                   43.376068
Equity Final [$]                 10102.277482
Equity Peak [$]                  10291.717912
Return [%]                           1.022775
Buy & Hold Return [%]               -7.035947
Return (Ann.) [%]                    7.382881
Volatility (Ann.) [%]               18.275091
Sharpe Ratio                         0.403986
Sortino Ratio                        0.729573
Calmar Ratio                         1.759983
Max. Drawdown [%]                   -4.194859
Avg. Drawdown [%]                   -0.888331
Max. Drawdown Duration        8 days 02:45:00
Avg. Drawdown Duration        1 days 05:45:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                       1.008354
Worst Trade [%]                     -0.527645
Avg. Trade [%]                    

In [8]:
stats = bt.optimize(
    roll=range(10, 60, 5),
    maximize="Equity Final [$]",
    constraint=lambda p: p.roll > 10,
)
stats

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.9/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Start                     2023-01-17 09:30:00
End                       2023-03-08 15:45:00
Duration                     50 days 06:15:00
Exposure Time [%]                    4.807692
Equity Final [$]                 10149.401675
Equity Peak [$]                  10149.401675
Return [%]                           1.494017
Buy & Hold Return [%]               -7.035947
Return (Ann.) [%]                   10.938703
Volatility (Ann.) [%]                3.333441
Sharpe Ratio                         3.281505
Sortino Ratio                             inf
Calmar Ratio                         8.567644
Max. Drawdown [%]                   -1.276746
Avg. Drawdown [%]                   -0.428759
Max. Drawdown Duration        0 days 04:30:00
Avg. Drawdown Duration        0 days 01:45:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       1.041394
Worst Trade [%]                       0.45102
Avg. Trade [%]                    

In [9]:
stats['_strategy']

<Strategy MeanReversion(roll=15)>

In [10]:
bt.plot()

Row(id='2368', ...)